In [1]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from numpy.testing import assert_almost_equal

from myfunc.my_roc_auc import my_roc_auc

# Scikit-learnと自作のAUC比較

## サンプルデータで一致確認

In [2]:
# 実装がsklearnのAUCと一致するかテスト

n_samples = 1000
np.random.seed(n_samples)

# generate sample data
true_label = np.random.choice([0,1], size=n_samples)
pred_proba = np.random.choice(np.linspace(0, 1, 100), size=n_samples)

In [3]:
# calc ROCAUC
roc_auc_sklearn = roc_auc_score(true_label, pred_proba)
roc_auc_mine = my_roc_auc(true_label, pred_proba)

print("ROCAUC\n\tsklearn:\t%.15f\n\tmine:\t%.15f"%(roc_auc_sklearn, roc_auc_mine))
assert_almost_equal(roc_auc_sklearn, roc_auc_mine, decimal=10)

ROCAUC
	sklearn:	0.523924202292845
	mine:	0.523924202292845


## assert_almost_equalの挙動確認

In [4]:
# 指定したprediction (decimal)での一致を確認できない場合Errorがでる

try:
    assert_almost_equal(roc_auc_sklearn, roc_auc_mine, decimal=17) # 小数点 16位までの一致を確認
except AssertionError:
    # raise
    print("AssertionError Raised")
    print("ROCAUC\n\tsklearn:\t%.16f\n\tmine:\t%.16f"%(roc_auc_sklearn, roc_auc_mine))

AssertionError Raised
ROCAUC
	sklearn:	0.5239242022928446
	mine:	0.5239242022928445


## 複数のデータに対しての一致を確認

In [5]:
# 実装がsklearnのAUCと一致するかテスト, エラーがでなければ一致している

n_trials = 1000
n_samples = 200

for n in tqdm(range(n_trials)):
    np.random.seed(n)
    # generate sample data
    true_label = np.random.choice([0,1], size=n_samples)
    pred_proba = np.random.choice(np.linspace(0, 1, 100), size=n_samples)
    
    # calc ROCAUC
    roc_auc_sklearn = roc_auc_score(true_label, pred_proba)
    roc_auc_mine = my_roc_auc(true_label, pred_proba)

    # check almost equal
    assert_almost_equal(roc_auc_sklearn, roc_auc_mine, decimal=10)

100%|██████████| 1000/1000 [00:12<00:00, 79.22it/s]


# 不均衡データにおいてのROC AUCの挙動確認

## すべて同じスコアが入る場合

In [6]:
# example 1
y_true_label = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0])
y_pred_proba = np.array([0.5] * 10)

In [7]:
# calc ROCAUC
roc_auc_sklearn = roc_auc_score(y_true_label, y_pred_proba)
roc_auc_mine = my_roc_auc(y_true_label, y_pred_proba)

print("ROCAUC\n\tsklearn:\t%.5f\n\tmine:\t%.5f"%(roc_auc_sklearn, roc_auc_mine))

ROCAUC
	sklearn:	0.50000
	mine:	0.50000


## 少数クラスのサンプルを1サンプル分類できるようになった場合の精度

In [8]:
# example 2
y_true_label =   np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0])
y_pred_proba = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0])

In [9]:
# calc ROCAUC
roc_auc_sklearn = roc_auc_score(y_true_label, y_pred_proba)
roc_auc_mine = my_roc_auc(y_true_label, y_pred_proba)

print("ROCAUC\n\tsklearn:\t%.5f\n\tmine:\t%.5f"%(roc_auc_sklearn, roc_auc_mine))

ROCAUC
	sklearn:	0.75000
	mine:	0.75000


## 多数クラスのサンプルを1サンプル分類できるようになった場合の精度

In [10]:
# example 3
y_true_label =   np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0])
y_pred_proba = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5])

In [11]:
# calc ROCAUC
roc_auc_sklearn = roc_auc_score(y_true_label, y_pred_proba)
roc_auc_mine = my_roc_auc(y_true_label, y_pred_proba)

print("ROCAUC\n\tsklearn:\t%.5f\n\tmine:\t%.5f"%(roc_auc_sklearn, roc_auc_mine))

ROCAUC
	sklearn:	0.56250
	mine:	0.56250


## ROCAUC上昇幅の比率を見る

In [12]:
# 1サンプルを分類成功することによるROC AUC上昇幅の比率を確認 
# (少数クラスの上昇幅) / (多数クラスの上昇幅)

(0.75000 - 0.5) / (0.56250 - 0.5)

4.0

In [13]:
# 不均衡データの比率を計算する
n_major = np.sum(y_true_label == 0)
n_minor = np.sum(y_true_label == 1)
minor_ratio = n_minor / n_major
print("少数クラスの比率 :\t\t\t %.3f"%minor_ratio)
print("少数クラスの比率の逆数 :\t %.3f"%(1/minor_ratio))

少数クラスの比率 :			 0.250
少数クラスの比率の逆数 :	 4.000


少数クラスの1サンプル分類成功することによるROC AUC上昇幅は、その少数クラスの比率の逆数倍だけ多数クラスの上昇幅より大きいことがわかる。
(ただし、予測スコアの不等号が完全に成立する場合)

つまり、ROC AUCはクラス比率の逆数によって、各クラス重みづけて評価していることがわかる。